# MCP ToolSpec

This tool connects to MCP Servers and allows an Agent to call the tools provided by MCP Servers.

This idea is migrated from [Integrate MCP tools into LlamaIndex](https://psiace.me/posts/integrate-mcp-tools-into-llamaindex/).
```bash
# get the code
git clone https://github.com/run-llama/llama_index
cd llama_index/llama-index-integrations/tools/llama-index-tools-mcp/examples


cp .env.example .env
# NOTE: edit the .env file to have the correct values!

# run the server
python mcp_server.py --server_type=sse
```


In this example, we will create a agent to search the web or retrieve the movie details using the CSV file.

It's built using the `AgentWorkflow` class from LlamaIndex. If that's new to you, you can [read more about it](https://docs.llamaindex.ai/en/stable/examples/agent/agent_workflow_basic/)!

In [16]:
from llama_index.llms.openai import OpenAI
import dotenv

dotenv.load_dotenv()

llm = OpenAI(model="gpt-4o")

In [21]:
from llama_index.tools.mcp import McpToolSpec
from llama_index.core.agent.workflow import FunctionAgent, ToolCallResult, ToolCall
from llama_index.core.workflow import Context

SYSTEM_PROMPT = """\
You are an AI assistant.

You have access to movie database
"""


async def get_agent(tools: McpToolSpec):
    tools = await tools.to_tool_list_async()
    agent = FunctionAgent(
        name="Agent",
        description="An agent that can fetch movie details and search web with user query.",
        tools=tools,
        llm=llm,
        system_prompt=SYSTEM_PROMPT,
    )
    return agent


async def handle_user_message(
    message_content: str,
    agent: FunctionAgent,
    agent_context: Context,
    verbose: bool = False,
):
    handler = agent.run(message_content, ctx=agent_context)
    async for event in handler.stream_events():
        if verbose and type(event) == ToolCall:
            print(f"Calling tool {event.tool_name} with kwargs {event.tool_kwargs}")
        elif verbose and type(event) == ToolCallResult:
            print(f"Tool {event.tool_name} returned {event.tool_output}")

    response = await handler
    return str(response)

In [34]:
from llama_index.tools.mcp import BasicMCPClient, McpToolSpec

# We consider there is a mcp server running on 127.0.0.1:8000, or you can use the mcp client to connect to your own mcp server.
mcp_client = BasicMCPClient("http://127.0.0.1:8000/sse")
mcp_tool = McpToolSpec(client=mcp_client)

# get the agent
agent = await get_agent(mcp_tool)

# create the agent context
agent_context = Context(agent)

In [35]:
# Run the agent!
while True:
    user_input = input("Search movies by title: ")
    if user_input == "exit":
        break
    print("User: ", user_input)
    response = await handle_user_message(user_input, agent, agent_context, verbose=True)
    print("Agent: ", response)

User:  what are the latest movies
Calling tool search_web with kwargs {'query': 'latest movies 2023'}
Tool search_web returned meta=None content=[TextContent(type='text', text='{"searchParameters": {"q": "latest movies 2023", "type": "search", "num": 2, "engine": "google"}, "organic": [{"title": "Movie, Release date between 2023-01-01 and 2023-12-31 ... - IMDb", "link": "https://www.imdb.com/year/2023", "snippet": "1. Oppenheimer \\u00b7 2. Hell of a Summer \\u00b7 3. The Hunger Games: The Ballad of Songbirds & Snakes \\u00b7 4. Anyone But You \\u00b7 5. Mission: Impossible - Dead Reckoning Part One.", "position": 1}, {"title": "Best Movies 2023 - Rotten Tomatoes", "link": "https://editorial.rottentomatoes.com/guide/best-2023-movies/", "snippet": "Best Movies 2023 \\u00b7 Oppenheimer (2023) Tomatometer icon \\u00b7 Mission: Impossible - Dead Reckoning Part One (2023) Tomatometer icon \\u00b7 Killers of the Flower Moon (2023)", "sitelinks": [{"title": "Best Wide Release Movie", "link": 

Here, we can see the agent is calling the `fetch_ipinfo` tool to get the ip info! This tool is running remotely on the mcp server.

The `MCPToolSpec` is connecting to the MCP server and creating `FunctionTool`s for each tool that is registered on the MCP server.

In [11]:
tools = await mcp_tool.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)

get_alerts Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    
get_forecast Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    


You can also limit the tools that the `MCPToolSpec` will create by passing a list of tool names to the `MCPToolSpec` constructor.

In [12]:
mcp_tool = McpToolSpec(client=mcp_client, allowed_tools=["some fake tool"])
tools = await mcp_tool.to_tool_list_async()
for tool in tools:
    print(tool.metadata.name, tool.metadata.description)